# 02. PyTorch Neural Network Classification

## Different types of `classifications`:
1. `Binary Classification`
   - yes/no
   - spam/not spam
3. `Multi-class Classification`
   - dog, cat, or chicken
5. `Multi-label Classification`
   - single input multiple outputs(hen,dog/cat,hen)

## Classification Inputs and Outputs
![](images/classification_input_output.png)

Shape of input: `[32,2,224,224]`